In [ ]:

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF




In [ ]:
start_date_str = "2023-07-01"
next_date = datetime.strptime(start_date_str, "%Y-%m-%d") + timedelta(days=1)
new_date_str = next_date.strftime("%Y-%m-%d")

In [ ]:
class statistical_model():
    def __init__(self):
        #self.client = gridstatusio.GridStatusClient(api_key = '9c97c76ced574dfdaf7b4a9618387599')
        self.kernel = RBF(length_scale=10, length_scale_bounds=(1e-2, 1e2))
        self.model = GaussianProcessRegressor(kernel=self.kernel, n_restarts_optimizer=10)

    def get_data_statusgrid(self, start_date:str, end_date:str) -> pd.DataFrame:
        self.lmps = self.client.get_dataset(
                                    dataset = "ercot_lmp_by_settlement_point",
                                    start = start_date,
                                    end = end_date,    
                                    timezone = "US/Central",
                                    )
        self.lmps.to_csv('summer_2023.cvs')
    
    def get_lmps_csv(self, file) -> pd.DataFrame:
        self.lmps = pd.read_csv(file)
        self.lmps.set_index('Date', inplace= True, drop=True)
        self.lmps = self.preprocess_data(self.lmps)

    def get_demand_forecast_csv(self, file) -> pd.DataFrame:
        self.demand_forecast = pd.read_csv(file)
        self.demand_forecast.set_index('Date', inplace= True, drop=True)
        self.demand_forecast = self.preprocess_data(self.demand_forecast)



    def preprocess_data(self, df):

        df.dropna(inplace = True)
        return df

        
    def train_model(self):
        self.model.fit(self.demand_forecast, self.lmps)
    
    def forecast(self, x):
        
        pred, st_dev = self.model.predict(x)
        return pred, st_dev
    
    
        

In [ ]:
first_model = statistical_model()
first_model.get_lmps_csv("data/nodal_lmps.csv")
first_model.get_demand_forecast_csv("data/zonal_demand_forecast.csv")
pred, std = first_model.train_model()
print(std)

In [ ]:
x = first_model.demand_forecast.iloc[0:2]
first_model.forecast(x)